In [0]:
from pyspark.sql.functions import count

In [0]:
def drop_duplicates(df, group_by):
    duplicates = df.groupBy(group_by).count().filter("count > 1")
    if duplicates.count() > 0:
        new_df = df.dropDuplicates(group_by)
        return new_df
    else:
        return df

In [0]:
from delta.tables import DeltaTable

In [0]:
def merge_delta_data(catalog,schema,table, input_df, merge_condition, partition_column=None):
    if(spark._jsparkSession.catalog().tableExists(f"{catalog}.{schema}.{table}")):
        delta_table = DeltaTable.forName(spark, f"{catalog}.{schema}.{table}")
        delta_table.alias("tgt").merge(
            input_df.alias("src"),
            merge_condition) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{catalog}.{schema}.{table}")


In [0]:
from pyspark.sql.functions import col, year, month, dayofweek, dayofmonth, date_format

In [0]:
def enrich_date_columns(df):
    df = df.withColumn("month", date_format(col("occurrence_date"), "MMMM"))
    df = df.withColumn("day", date_format(col("occurrence_date"), "EEEE"))
    df = df.withColumn("is_weekend", (dayofweek(col("occurrence_date"))).isin([1, 7]))
    return df

In [0]:
def enrich_occurrence_columns(df):
    df = df.withColumn("total_casualties", df.casualties_crew + df.casualties_pax + df.casualties_ground)
    return df